# Origami Frame Evolution

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys
from tqdm import tqdm
from matplotlib.patches import Circle
from matplotlib import cm

In [ ]:
# Define your circle centers and radius
circle_centers = [(-30, -20), (-30, 0), (-30, 20),
                  (-10, -20), (-10, 0), (-10, 20),
                  (10, -20), (10, 0), (10, 20),
                  (30, -20), (30, 0), (30, 20)]
radius = 10

In [ ]:
imagers = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12']
hues = np.arange(0, 1, 1 / 12)
colors = [colorsys.hsv_to_rgb(_, 1, 1) for _ in hues]
colors = colors[-6:] + colors[:-6]
color_map = dict(zip(imagers, colors))

In [ ]:
# Define File Location
# file = '/Volumes/DataVol4/Micky_tmp/Origami_Data/20251031_R1/300mW_150Picks_R1_avg_avg3_picked_20_avg3.hdf5'
files = ['/Volumes/DataVol4/Micky_tmp/Origami_Data/20251030_M2/500pM/300mW_150Picks_M2_avg_avg3_picked_20_avg3.hdf5', '/Volumes/DataVol4/Micky_tmp/Origami_Data/20251031_R1/300mW_150Picks_R1_avg_avg3_picked_20_avg3.hdf5']    
folder = '/Volumes/DataVol4/Micky_tmp/Origami_Data/'

In [ ]:
def load_data(file):
    data = pd.read_hdf(file, key='locs')
    if 'R1' in file:
        imager = 'R1'
    elif 'M2' in file:
        imager = 'R8'
    data['x'] = data['x'] - data['x'].mean()
    data['y'] = data['y'] - data['y'].mean()
    data['x'] = data['x'] * 130  # nm
    data['y'] = data['y'] * 130  # nm
    return data, imager

def plot_evolution(data, imager, folder):
    # Count number of groups
    all_groups = sorted(data['group'].unique())
    n_total = len(all_groups)

    frame_ranges = [(0, (i+1)*1000) for i in range(10)]

    # Define frame ranges for 10 rows
    frame_ranges = [(0, (i+1)*1000) for i in range(10)]

    # Define batch size
    batch_size = 20


    # Define colormap for background (e.g. light colors for low activity, bright for many red circles)
    cmap = cm.get_cmap('Reds')
    max_circles = len(circle_centers)  # 12

    # Loop through groups in batches of 10
    for start in range(0, n_total, batch_size):
        groups = all_groups[start:start + batch_size]
        n_groups = len(groups)
        
        # Create grid for this batch
        fig, axes = plt.subplots(10, n_groups, figsize=(3*n_groups, 3*10), sharex=True, sharey=True)
        
        if n_groups == 1:
            axes = axes.reshape(10, 1)
        
        for row_idx, (fmin, fmax) in enumerate(reversed(frame_ranges)):
            frame_subset = data[data['frame'].between(fmin, fmax)]
            for col_idx, group in enumerate(groups):
                ax = axes[row_idx, col_idx]
                group_subset = frame_subset[frame_subset['group'] == group]
                
                # Subsample for speed
                sample = group_subset.sample(min(2000, len(group_subset)), random_state=0)

                # --- Count filled circles ---
                filled_count = 0
                for (cx, cy) in circle_centers:
                    distances = np.sqrt((sample['x'] - cx)**2 + (sample['y'] - cy)**2)
                    count = np.sum(distances <= radius)
                    if count > threshold:
                        filled_count += 1
            
                # --- Compute background color from filled_count ---
                # Normalize count (0 to 1) for colormap
                bg_color = cmap(filled_count / max_circles)
                ax.set_facecolor(bg_color)
                
                # Plot scatter
                ax.scatter(sample['x'], sample['y'], s=3, rasterized=True, color='k')

                # after ax.scatter(...)
                for (cx, cy) in circle_centers:
                    distances = np.sqrt((sample['x'] - cx)**2 + (sample['y'] - cy)**2)
                    count = np.sum(distances <= radius)
                    if count > threshold:
                        color = 'red'
                        fill = True
                        # alpha = 0.7
                    else:
                        color = 'black'
                        fill = False
                        # alpha = 0.0
                    ax.add_patch(Circle((cx, cy), radius, fill=fill, color=color, lw=1))
                # ax.hist2d(sample['x'], sample['y'], bins=120, range=[[-60, 60], [-60, 60]], cmap='Blues', cmin=1)

                ax.set_xlim(-60, 60)
                ax.set_ylim(-60, 60)
                ax.set_aspect('equal', 'box')
                ax.set_xticks([])
                ax.set_yticks([])
                ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)


                # if row_idx == 0:
                #     ax.set_title(f'Group {group}', fontsize=9)
                # if col_idx == 0:
                #     ax.set_ylabel(f'≤ {fmax}', fontsize=8)
        
        plt.tight_layout()
        plt.savefig(f'{folder}/{imager}_evolution.svg', format='svg', bbox_inches='tight')
        plt.show()

def compute_cumulative_coverage(data, imager):
    data = data.sort_values("frame")

    all_groups = sorted(data["group"].unique())
    results = []

    # --- Compute per-group cumulative coverage ---
    for group in tqdm(all_groups, desc="Processing groups"):
        subset = data[data["group"] == group]
        frames = np.sort(subset["frame"].unique())

        cumulative_data = pd.DataFrame(columns=["x", "y"])
        group_results = []

        for f in frames:
            new_points = subset[subset["frame"] == f][["x", "y"]]
            cumulative_data = pd.concat([cumulative_data, new_points], ignore_index=True)

            # Count filled circles
            filled_count = 0
            for (cx, cy) in circle_centers:
                distances = np.sqrt((cumulative_data["x"] - cx)**2 + (cumulative_data["y"] - cy)**2)
                if np.sum(distances <= radius) > threshold:
                    filled_count += 1

            coverage = filled_count / len(circle_centers)
            group_results.append({"frame": f, "coverage": coverage})

        group_df = pd.DataFrame(group_results)
        group_df["group"] = group
        results.append(group_df)

    results_df = pd.concat(results, ignore_index=True)
    results_df["coverage"] = results_df.groupby("group")["coverage"].cummax()

    frames_all = np.arange(data["frame"].min(), data["frame"].max() + 1)
    aligned = (
    results_df.set_index("frame")
    .groupby("group")["coverage"]
    .apply(lambda s: s.reindex(frames_all).ffill().fillna(0))
    .reset_index()
    )
    agg_df = aligned.groupby("frame")["coverage"].agg(["mean", "std"]).reset_index()
    agg_df["smooth_mean"] = agg_df["mean"].rolling(window=100, min_periods=1).mean()
    agg_df["smooth_std"] = agg_df["std"].rolling(window=100, min_periods=1).mean()
    agg_df["imager"] = imager
    return agg_df

In [ ]:
threshold = 20
datasets = []

for file in files:
    data, imager = load_data(file)
    plot_evolution(data, imager, folder)
    agg_df = compute_cumulative_coverage(data, imager)
    datasets.append(agg_df)

In [ ]:
for df in datasets:
    imager = df["imager"].iloc[0]
    color = color_map[imager]
    
    plt.plot(df["frame"], df["smooth_mean"], color=color, lw=2, label=imager)
    plt.fill_between(df["frame"],
                     df["smooth_mean"] - df["smooth_std"],
                     df["smooth_mean"] + df["smooth_std"],
                     color=color, alpha=0.2)
    
plt.xlim(0, 10000)
plt.ylim(0, 1)

plt.xlabel("Frame")
plt.ylabel("Fraction of filled circles (cumulative)")
plt.title("Cumulative coverage comparison")
plt.legend()
plt.savefig(f'{folder}/cumulative_coverage_comparison.svg', format='svg', bbox_inches='tight')
plt.show()